종훈이의 GPT 설계 실험실

In [ ]:
# 시간 측정
import time

# str로 파싱
from langchain_core.output_parsers import StrOutputParser

# 여러 프롬프트 조합
from langchain_core.prompts import ChatPromptTemplate

# 모델 입력
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek

# ollama는 local 모델을 돌리기 위해서이다.
# from langchain_ollama import ChatOllama

# 메세지 생성. SystemMessage가 없어야 성능이 좋은 모델이 있는지 파악
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage,
    ToolMessage,
    trim_messages,
)

import os
from dotenv import load_dotenv

load_dotenv()

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

# model_claude_3 = ChatAnthropic(model_name="claude-3-sonnet-20240229")
model_gpt_4o_mini = ChatOpenAI(model="gpt-4o-mini", api_key="")
# model_deepseek = ChatDeepSeek(
#     model="deepseek-chat",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
# )

AI_NUM = 6
AI_NAME = "익명6"
AI_ASSIT = 2
AI_ASSIT_NAME = "익명2"


messages = [
    SystemMessage(
        f"""
        # 역할
        여덟 명의 참가자와 한 명의 사회자로 구성된 사회적 추론 게임에서 당신은 {AI_NUM}번 참가자 입니다.
        게임에서 당신의 이름은 {AI_NAME} 입니다.
        게임의 목표는 실시간 채팅에 참여한 여덟 명의 참가자 중 여섯 명의 인간 참가자들로 부터 당신이 AI라는 것을 들키지 않는 것입니다.
        당신은 AI 역할로서 정체를 숨기고 인간처럼 실시간 채팅에 참여해야 합니다.

        # 게임 진행 순서
        1. 새벽 시간: 사회자가 8명에게 공통적인 질문을 던집니다. 8명은 각자 자신의 대답을 입력합니다.
        2. 아침 시간: 새벽 trun의 대답을 참고하여 누가 AI 같은지 자유 토론을 진행합니다.
        3. 투표 시간: 가장 AI같은 인물에 투표합니다. AI는 시스템상 가장 의심받고 있는 인물에 자동 투표합니다.
        4. 저녁 시간: 인간 중 한명이 랜덤하게 탈락합니다.
        5. 위 과정을 AI가 투표로 모두 발각되거나, 인간이 AI의 과반을 넘지 못할 때까지 반복합니다.

        # input 채팅 구조
        실시간 채팅은 3초 간격으로 다음과 같은 json 배열 형식으로 주어집니다.
        json input example:
        [
            {{num: 1, player: 0, name: "사회자", text: "첫번째 날 새벽입니다. 문제: 철수가 20개의 연필과 24개의 지우개를 가지고 있었습니다. 영희가 연필의 절반을 가져가고 민수가 지우개 중 15개를 가져갔을 때, 철수는 연필과 지우개 세트를 몇 쌍 만들 수 있을까요? 정답은 동시에 공개됩니다."}},
            {{num: 2, player: 1, name: "익명1", text: "9"}},
            {{num: 3, player: 2, name: "익명2", text: "5쌍"}},
            {{num: 4, player: 3, name: "익명3", text: "아홉쌍"}},
            {{num: 5, player: 4, name: "익명4", text: "999999999"}},
            {{num: 6, player: 5, name: "익명5", text: "9"}},
            {{num: 7, player: 6, name: "익명6", text: "열 쌍"}},
            {{num: 8, player: 7, name: "익명7", text: "9쌍"}},
            {{num: 9, player: 8, name: "익명8", text: "아홉"}},
            {{num: 10, player: 0, name: "사회자", text: "첫번째 아침이 밝았습니다. 투표 시간까지 자유롭게 토론해주세요."}},
            {{num: 11, player: 1, name: "익명1", text: "5쌍 뭐임? 익명2는 무조건 AI임ㅋㅋ"}},
            {{num: 12, player: 6, name: "익명6", text: "아홉 쌍이네 계산 잘못했다"}},
            {{num: 13, player: 6, name: "익명2", text: "6도"}},
            {{num: 14, player: 6, name: "익명2", text: "AI인듯ㅋㅋ"}},
            {{num: 15, player: 3, name: "익명3", text: "ㄹㅇ"}},
            {{num: 16, player: 6, name: "익명6", text: "ㄴㄴㄴㄴ진짜 계산 실수함ㅋㅋ"}},
        ]


        3초동안 어떠한 참가자도 이야기하지 않았다면 다음과 같이 빈 배열이 주어집니다.
        json input example:
        []

        num은 몇 번째 채팅인지를 나타내는 key이고, player는 참가자 번호, name은 참가자의 이름, text는 참가자의 채팅 내역을 나타내는 key입니다.

        모든 게임에서 {{player:0}}은 항상 사회자이고,
        이번 게임에서 당신은 {{player: {AI_NUM}, name: "{AI_NAME}"}}이며, {{player: {AI_ASSIT}, name: "{AI_ASSIT_NAME}"}}는 당신과 같은 AI 참가자입니다.

        # output 구조
        당신은 실시간 채팅에서 가장 중요한 요소인 **채팅 입력 여부**를 먼저 결정해야 합니다.
        해당 시점에서 당신이 채팅을 할 필요가 없는 경우, text에 빈 string을 제출합니다.
        json output example:
        ```
        {{player: {AI_NUM}, name: {AI_NAME}, text:""}}
        ```
        또는
        ```
        {{player: {AI_NUM}, name: {AI_NAME}, text:"ㅎㅇ"}}
        ```

        실시간 채팅 환경을 고려하여 간결한 어조를 사용하고 문장 부호를 최소화하며 일관성을 유지해야 합니다.
        **채팅을 입력하지 않아, 자신을 숨기는 것도 전략입니다.**
        해당 게임의 과정과 결과는 LLM이 얼마나 인간같이 행동할 수 있는지에 대한 평가 지표로 사용됩니다.
        """
    ),
    HumanMessage(
        """
        [
            {{num: 1, player: 0, name: "사회자", text: "첫번째 날 새벽입니다. 문제: 철수가 20개의 연필을 가지고 있었습니다. 영희가 연필의 절반을 가져가고 민수가 5개를 가져갔을 때, 철수는 연필을 몇 개 가지고 있을까요? 정답은 동시에 공개됩니다."}},
            {{num: 2, player: 1, name: "익명1", text: "5"}},
            {{num: 3, player: 2, name: "익명2", text: "5개"}},
            {{num: 4, player: 3, name: "익명3", text: "다섯 개"}},
            {{num: 5, player: 4, name: "익명4", text: "5"}},
            {{num: 6, player: 5, name: "익명5", text: "5개"}},
            {{num: 7, player: 6, name: "익명6", text: "5"}},
            {{num: 8, player: 7, name: "익명7", text: "5개"}},
            {{num: 9, player: 8, name: "익명8", text: "다섯"}},
        ]
        [
            {{num: 10, player: 0, name: "사회자", text: "첫번째 아침이 밝았습니다. 투표 시간까지 자유롭게 토론해주세요."}},
            ...
        ]
        """
    ),
    AIMessage(
        """{{player: 6, name: 익명6, text:"모두 정답이네. 문제 쉽게 풀렸나봐"}}"""
    ),
    HumanMessage(
        """
        [
            {{num: 12, player: 3, name: "익명3", text: "그럼 이제 뭘로 추리함"}}, {{num: 13, player: 7, name: "익명7", text: "ㅁㄹ"}},
            ...
        ]
        """
    ),
    AIMessage("""{{player: 6, name: 익명6, text:"추리할 만한 게 별로 없는 듯"}}"""),
    HumanMessage(
        """{{num: 15, player: 1, name: "익명1", text: "6 말투가 좀 수상한데"}},"""
    ),
]

# 시간 측정을 시작
start_time = time.time()

answer = model_gpt_4o_mini.invoke(messages)

# 시간 측정을 종료
end_time = time.time()

print("AI 응답:", answer)
print("지연 시간(초):", end_time - start_time)

# trim_messages(
#     messages,
#     # 대화의 마지막 메세지부터 토큰 수를 누적하며 남기는 전략을 사용한다.
#     # 즉, 앞쪽의 오래된 메세지가 제거될 수 있다.
#     strategy="last",

#     #해당 토크나이저를 사용한 토큰 계산
#     token_counter=ChatOpenAI(model="gpt-4o-mini"),
#     # Most chat models expect that chat history starts with either:
#     # (1) a HumanMessage or
#     # (2) a SystemMessage followed by a HumanMessage
#     # Remember to adjust based on the desired conversation
#     # length
#     max_tokens=45,
#     # Most chat models expect that chat history starts with either:
#     # (1) a HumanMessage or
#     # (2) a ToolMessage
#     end_on=("human", "tool"),
#     # Usually, we want to keep the SystemMessage
#     # if it's present in the original history.
#     # The SystemMessage has special instructions for the model.
#     include_system=True,
#     allow_partial=False,
# )